# sklearn.preprocessing 方法总结

## StandardScaler

$$
\frac{X-\mu}{\sigma}
$$

In [1]:
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()

1. 首先拟合数据

In [2]:
scaler.fit(data)

StandardScaler(copy=True, with_mean=True, with_std=True)

2. 拟合后可以查看，拟合后的均值和方差

In [3]:
scaler.mean_

array([0.5, 0.5])

In [4]:
scaler.var_

array([0.25, 0.25])

3. 根据拟合，你转换新的数据

In [5]:
scaler.transform(data)

array([[-1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.]])

检验是否正确：
$$
[ \frac{2-0.5}{0.5} = 3 , \frac{2-0.5}{0.5} = 3]
$$

In [6]:
scaler.transform([[2, 2]])

array([[3., 3.]])

## MinMaxScaler

$$
\frac{X-X.min}{X.max-X.min} 的范围在（0，1）
$$

In [28]:
from sklearn.preprocessing import MinMaxScaler
data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
scaler = MinMaxScaler()

# 属性值如下：

print(scaler.fit(data))
print(scaler.data_max_)
print(scaler.data_min_)
print(scaler.data_range_)

MinMaxScaler(copy=True, feature_range=(0, 1))
[ 1. 18.]
[-1.  2.]
[ 2. 16.]


## OneHotEncoder 

onehot 编码：可以分为两类，通过句柄控制

- hanlde_unknown

In [29]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

默认编码方式，通过categories查看，按照顺序排序编码，如下：第一位是：Female,第二维使'Male'....

In [30]:
enc.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [31]:
enc.transform([['Female', 1], ['Male', 4]]).toarray()

array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [32]:
enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

array([['Male', 1],
       [None, 2]], dtype=object)

- drop 参数编码时可以丢掉某一列：

In [33]:
drop_enc = OneHotEncoder(drop='first').fit(X)
print(drop_enc.categories_)
print(drop_enc.transform([['Female', 1], ['Male', 2]]).toarray())

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
[[0. 0. 0.]
 [1. 1. 0.]]


## Normalizer：范数归一化

- Normalizer(norm='l2', copy=True)
    - 需要传入的参数norm: 
        - 'l1'
        - 'l2'
        - 'max'

向量$x(x_1,x_2,\dots,x_n)$的 $L^2$范数的定义为：

$$
norm(x)=\sqrt(x_1^2+x_2^2+\dots+x_n^2)
$$

要使得x归一化到单位$L^2$范数，即建立一个从x到$x'$的映射，使得$x'$的$L^2$范数为1

则：
$$
\begin{aligned}
1 &=\operatorname{norm}\left(\mathrm{x}^{\prime}\right)=\frac{\sqrt{x_{1}^{2}+x_{2}^{2}+\ldots+x_{n}^{2}}}{\operatorname{norm}(x)} \\
&=\sqrt{\frac{x_{1}^{2}+x_{2}^{2}+\ldots+x_{n}^{2}}{\operatorname{norm}(x)^{2}}} \\
&=\sqrt{\left(\frac{x_{1}}{\operatorname{norm}(x)}\right)^{2}+\left(\frac{x_{2}}{\operatorname{norm}(x)}\right)^{2}+\ldots+\left(\frac{x_{n}}{\operatorname{norm}(x)}\right)^{2}} \\
&=\sqrt{x_{1}^{2}+x_{2}^{\prime 2}+\ldots+x_{n}^{\prime 2}}
\end{aligned}
$$

即：
$$
x_{i}^{\prime}=\frac{x_{i}}{\operatorname{nom}(x)}
$$




In [55]:
from sklearn.preprocessing import Normalizer
X = [[4, 1, 2, 2],
     [1, 3, 9, 3],
     [5, 7, 5, 1]]
transformer = Normalizer().fit(X)  # fit does nothing.
transformer.transform(X)

array([[0.8, 0.2, 0.4, 0.4],
       [0.1, 0.3, 0.9, 0.3],
       [0.5, 0.7, 0.5, 0.1]])

计算L2的范数：$\sqrt(4^2+1+4+4)=5$ ,这里就是计算L2范数空间，即欧拉空间中的最大值。

In [50]:
transformer.get_params

<bound method BaseEstimator.get_params of Normalizer(copy=True, norm='l2')>

In [51]:
transformer.set_params(norm='l1')

Normalizer(copy=True, norm='l1')

## normalize函数

这个函数的作用和上面介绍的Normalizer()的功能一样，换句话说函数`normalize(X,norm='l2')`等价于`sklearn.preprocessing.Normalizer().fit(X)`

注：这只是一个函数，没有可调用的方法，当然也不可以作为一个管道，放到Pipeline中，所以不推荐。

In [68]:
from sklearn.preprocessing import  normalize
normalize(X)

array([[0.8, 0.2, 0.4, 0.4],
       [0.1, 0.3, 0.9, 0.3],
       [0.5, 0.7, 0.5, 0.1]])

# preprocessing的通用方法介绍

In [7]:
scaler.get_params(deep=True)

{'copy': True, 'with_mean': True, 'with_std': True}

同时拟合和转换数据：fit_transform() 结果和上头的 fit() 和 transform()

In [8]:
a = scaler.fit_transform(data)
a

array([[-1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.]])

In [9]:
scaler.inverse_transform(a)

array([[0., 0.],
       [0., 0.],
       [1., 1.],
       [1., 1.]])

In [23]:
print(scaler.set_params(with_std=False))
print(scaler.fit_transform(data))
print(scaler.mean_)
print(scaler.var_)

StandardScaler(copy=True, with_mean=True, with_std=False)
[[-0.5 -0.5]
 [-0.5 -0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]]
[0.5 0.5]
None
